In [ ]:
import pandas as pd
import pickle
import csv 
import numpy as np


# Read the csv file 
df = pd.read_csv('chordonomicon.csv')


# Define the file path for the mirex mapping
extensions_file_path = 'mirex_mapping.txt'

In [ ]:
# Initialize an empty list to store the tuples
data_list = []

# Open the extensions file for reading
with open(extensions_file_path, 'r') as ext_file:
    # Iterate through each line in the extensions file
    for line in ext_file:
        # Split the line into two values (columns)
        columns = line.strip().split()

        # Check if there are exactly two columns
        if len(columns) == 2:
            # Create a tuple from the columns and append it to the list
            data_list.append((columns[0], columns[1]))

In [ ]:
def find_inversion(root, base):
    sharp_pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
    flat_pitches = ['C','Db','D','Eb','E','F','Gb','G','Ab','A','Bb','B']
    strange_pitches = ['B#','C#','D','D#','Fb','E#','F#','G','G#','A','A#','Cb']
    sharp_inversions = ['1','1#','2','2#','3','4','4#','5','5#','6','6#','7']
    flat_inversions = ['1','2b','2','3b','3','4','5b','5','6b','6','7b','7']
    if root in sharp_pitches:
        root_position = sharp_pitches.index(root)
    elif root in flat_pitches:
        root_position = flat_pitches.index(root)
    else:
        root_position = strange_pitches.index(root)
    if base in sharp_pitches:
        base_position = sharp_pitches.index(base)
    elif base in flat_pitches:
        base_position = flat_pitches.index(base)
    else:
        base_position = strange_pitches.index(base)
    inversion_position = (root_position - base_position)
    if inversion_position > 0:
        inversion_position = 12 - inversion_position
    else:
        inversion_position = abs(inversion_position)
    if '#' in base:
        return(sharp_inversions[inversion_position])
    elif 'b' in base:
        return(flat_inversions[inversion_position])
    else:
        return(sharp_inversions[inversion_position])
    

c = 0
mirex_list = []
for i in range(len(df['chords'])):
    converted_string = ''
    file = df['chords'][i]
    chords = file.split()
    c += 1
    if c % 1000 == 0:
        print(c)
    for whole_chord in chords:
        chord = whole_chord.partition('/')[0]
        base = whole_chord.partition('/')[2].replace('s','#')
        if '<' not in chord:
            mirex_chord = 'wtf'
            if len(chord) == 1:
                mirex_chord = chord + ':maj'
                root_chord = chord
            elif len(chord) == 2:
                if (chord[1] == 's') or (chord[1] == 'b'):
                    chord = chord.replace('s', '#')  # Fix: replace returns the modified string
                    mirex_chord = chord + ':maj'
                    root_chord = chord
                else:
                    mirex_chord = chord[0] + ':' + chord[1]
                    root_chord = chord[0]
            else:
                if ((chord[1] == 's') and (chord[2] != 'u')) or (chord[1] == 'b'):
                    chord = chord[:1] + "#" + chord[2:]
                    root_chord = chord[0] + chord[1]
                else:
                    root_chord = chord[0]
                extension = chord.replace(root_chord, '')
                for tuple in data_list:
                    if extension == tuple[0]:
                        mirex_chord = root_chord + ':' + tuple[1]
                        break
                    else:
                        # add "fal" to check for bugs
                        mirex_chord = 'fal'
            if mirex_chord == 'fal':
                print(chord)
            if '/' in whole_chord and base !='':
                mirex_chord = mirex_chord + '/' + find_inversion(root_chord, base)
            # Write the processed chord to the output file
            converted_string += mirex_chord + ' '
        else:
            converted_string += chord + ' '
    # Write a newline character after processing each line
    mirex_list.append(converted_string)